In [ ]:
from yugiquery import *
header('Bandai')

---

# Data aquisition

## Fetch online data

In [ ]:
timestamp = pd.Timestamp.now()

In [ ]:
bandai_df = fetch_bandai()

# Check changes

## Load previous data

In [ ]:
# Get list of files
files_list = sorted(glob.glob('../data/all_bandai_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if files_list:
    latest_file = files_list[0]
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(latest_file, dtype=object, keep_default_na=False, na_values=[''])
    # Force dtypes to match current df
    previous_df = previous_df.astype(bandai_df[previous_df.columns.intersection(bandai_df.columns)].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(latest_file).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

## Generate changelog

In [ ]:
if previous_df is None:
    changelog=None
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, bandai_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/bandai_changelog_{previous_ts.isoformat(timespec="minutes")}_{timestamp.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print('No changes. New data not saved')
else:
    bandai_df.to_csv(f'../data/all_bandai_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
    print('Data saved')

# Data visualization

In [ ]:
bandai_df

## Card type

In [ ]:
print('Total number of card types:', bandai_df['Card type'].nunique())

In [ ]:
card_type_colors = [colors_dict[i] for i in bandai_df['Card type'].value_counts().index]
bandai_df['Card type'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=0, color = card_type_colors, title = 'Card type')
plt.show()

## ATK

In [ ]:
print('Total number of ATK values:', bandai_df['ATK'].nunique())

In [ ]:
bandai_df.drop(columns=['Card type', 'Page name', 'Page URL']).groupby('ATK').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
atk_colors = colors_dict['Monster Card']
bandai_df['ATK'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, color = atk_colors, title = 'ATK')
plt.show()

## DEF

In [ ]:
print('Total number of DEF values:', bandai_df['DEF'].nunique())

In [ ]:
bandai_df.drop(columns=['Card type', 'Page name', 'Page URL']).groupby('DEF').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
def_colors = colors_dict['Monster Card']
bandai_df['DEF'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, color = def_colors, title = 'DEF')
plt.show()

## Level

In [ ]:
bandai_df.drop(columns=['Card type', 'Page name', 'Page URL']).groupby('Level').nunique().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
level_colors = colors_dict['Level']
bandai_df['Level'].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors = 'coerce')).plot.bar(figsize = (18,6), grid = True, rot=0, color= level_colors, title = 'Level/Rank')
plt.show()

### ATK statistics

In [ ]:
bandai_df[['Level','ATK']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Level').describe()

### DEF statistics

In [ ]:
bandai_df[['Level','DEF']].apply(pd.to_numeric, errors = 'coerce').dropna().astype(int).groupby('Level').describe()

## Monster type

In [ ]:
print('Total number of monster types:', bandai_df['Monster type'].nunique())

In [ ]:
bandai_df.drop(columns=['Card type', 'Page name', 'Page URL']).groupby('Monster type').nunique()

In [ ]:
monster_type_colors = colors_dict['Monster Card']
bandai_df['Monster type'].value_counts().plot.bar(figsize = (18,6), grid = True, rot=45, color = monster_type_colors, title = 'Monster type')
plt.show()

## Set

In [ ]:
print('Total number of sets:', bandai_df['Set'].nunique())

In [ ]:
bandai_df.drop(columns=['Page name', 'Page URL']).groupby('Set').nunique()

# Epilogue

In [ ]:
benchmark('bandai',timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Bandai.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Bb]andai*"

In [ ]:
! git commit -m {"'Bandai update-" + timestamp.isoformat() + "'"}